# 第 7 章：類別與物件導向

本章節詳細說明 Python 的類別定義、繼承機制、存取控制，以及特殊方法（Magic Methods）的使用。

---

## 7.1 類別定義

### 基本類別定義

**JavaScript 的類別：**
```javascript
class Person {
    constructor(name, age) {
        this.name = name;
        this.age = age;
    }

    greet() {
        return `Hello, I'm ${this.name}`;
    }
}

const person = new Person("Alice", 30);
```

**Python 的類別：**

In [ ]:
class Person:
    def __init__(self, name, age):
        self.name = name
        self.age = age

    def greet(self):
        return f"Hello, I'm {self.name}"

# 建立實例（不需要 new）
person = Person("Alice", 30)
print(person.greet())  # "Hello, I'm Alice"
print(person.name)     # "Alice"
print(person.age)      # 30

### 主要語法差異

| 特性 | JavaScript | Python |
|------|------------|--------|
| 建構子 | `constructor()` | `__init__()` |
| 實例參考 | `this` | `self`（必須明確宣告） |
| 建立實例 | `new ClassName()` | `ClassName()` |
| 方法定義 | `method() {}` | `def method(self):` |

### __init__ 方法（建構子）

In [ ]:
class Person:
    def __init__(self, name, age, city="Unknown"):
        """初始化方法，建立實例時自動呼叫"""
        self.name = name      # 實例屬性
        self.age = age
        self.city = city

# 建立實例
alice = Person("Alice", 30)
bob = Person("Bob", 25, "Taipei")

print(f"alice.name: {alice.name}")
print(f"alice.city: {alice.city}")  # 使用預設值
print(f"bob.city: {bob.city}")

In [ ]:
# 修改屬性
alice.age = 31
alice.email = "alice@example.com"  # 動態新增屬性

print(f"alice.age: {alice.age}")
print(f"alice.email: {alice.email}")

### self 參數

Python 中所有實例方法的第一個參數都是 `self`，代表實例本身：

In [ ]:
class Calculator:
    def __init__(self, value=0):
        self.value = value

    def add(self, n):
        self.value += n
        return self  # 回傳 self 可以實現鏈式呼叫

    def subtract(self, n):
        self.value -= n
        return self

    def multiply(self, n):
        self.value *= n
        return self

    def get_value(self):
        return self.value

# 鏈式呼叫
calc = Calculator(10)
result = calc.add(5).multiply(2).subtract(10).get_value()
print(f"(10 + 5) * 2 - 10 = {result}")

### 類別屬性 vs 實例屬性

In [ ]:
class Dog:
    # 類別屬性（所有實例共享）
    species = "Canis familiaris"
    count = 0

    def __init__(self, name, age):
        # 實例屬性（每個實例獨立）
        self.name = name
        self.age = age
        Dog.count += 1  # 修改類別屬性

# 類別屬性可以透過類別或實例存取
print(f"Dog.species: {Dog.species}")

dog1 = Dog("Buddy", 3)
dog2 = Dog("Max", 5)

print(f"dog1.species: {dog1.species}")
print(f"dog2.species: {dog2.species}")
print(f"Dog.count: {Dog.count}")

In [ ]:
# 透過實例修改會建立實例屬性，不影響類別屬性
dog1.species = "Modified"
print(f"dog1.species: {dog1.species}")  # 實例屬性
print(f"dog2.species: {dog2.species}")  # 類別屬性
print(f"Dog.species: {Dog.species}")    # 類別屬性

In [ ]:
# 注意：可變的類別屬性可能導致意外
class BadExample:
    items = []  # 危險！所有實例共享同一個列表

    def add_item(self, item):
        self.items.append(item)

a = BadExample()
b = BadExample()
a.add_item("x")
print(f"b.items: {b.items}")  # ["x"] - b 也受影響了！

In [ ]:
# 正確做法：在 __init__ 中初始化可變屬性
class GoodExample:
    def __init__(self):
        self.items = []  # 每個實例有自己的列表

    def add_item(self, item):
        self.items.append(item)

a = GoodExample()
b = GoodExample()
a.add_item("x")
print(f"a.items: {a.items}")  # ["x"]
print(f"b.items: {b.items}")  # []

### 類別方法與靜態方法

In [ ]:
class MyClass:
    class_var = "類別變數"

    def __init__(self, value):
        self.instance_var = value

    # 實例方法：第一個參數是 self（實例）
    def instance_method(self):
        return f"實例方法，存取 {self.instance_var}"

    # 類別方法：第一個參數是 cls（類別）
    @classmethod
    def class_method(cls):
        return f"類別方法，存取 {cls.class_var}"

    # 靜態方法：沒有自動傳入的參數
    @staticmethod
    def static_method(x, y):
        return f"靜態方法，計算 {x} + {y} = {x + y}"

# 使用方式
obj = MyClass("實例值")

# 實例方法：透過實例呼叫
print(obj.instance_method())

# 類別方法：透過類別或實例呼叫
print(MyClass.class_method())
print(obj.class_method())

# 靜態方法：透過類別或實例呼叫
print(MyClass.static_method(3, 5))
print(obj.static_method(3, 5))

### 類別方法的實際應用：替代建構子

In [ ]:
class Date:
    def __init__(self, year, month, day):
        self.year = year
        self.month = month
        self.day = day

    def __str__(self):
        return f"{self.year}-{self.month:02d}-{self.day:02d}"

    # 替代建構子（工廠方法）
    @classmethod
    def from_string(cls, date_string):
        """從字串建立 Date 物件"""
        year, month, day = map(int, date_string.split("-"))
        return cls(year, month, day)

    @classmethod
    def today(cls):
        """建立今天的 Date 物件"""
        import datetime
        today = datetime.date.today()
        return cls(today.year, today.month, today.day)

# 使用不同方式建立物件
date1 = Date(2024, 1, 15)
date2 = Date.from_string("2024-06-20")
date3 = Date.today()

print(f"date1: {date1}")
print(f"date2: {date2}")
print(f"date3: {date3}")

### 靜態方法的實際應用：工具類別

In [ ]:
class MathUtils:
    """數學工具類別"""

    @staticmethod
    def is_prime(n):
        """檢查是否為質數"""
        if n < 2:
            return False
        for i in range(2, int(n ** 0.5) + 1):
            if n % i == 0:
                return False
        return True

    @staticmethod
    def gcd(a, b):
        """計算最大公因數"""
        while b:
            a, b = b, a % b
        return a

    @staticmethod
    def lcm(a, b):
        """計算最小公倍數"""
        return abs(a * b) // MathUtils.gcd(a, b)

# 使用
print(f"is_prime(17): {MathUtils.is_prime(17)}")
print(f"gcd(48, 18): {MathUtils.gcd(48, 18)}")
print(f"lcm(4, 6): {MathUtils.lcm(4, 6)}")

### @property 裝飾器

將方法包裝成屬性存取，提供 getter、setter 和 deleter：

In [ ]:
import math

class Circle:
    def __init__(self, radius):
        self._radius = radius  # 使用底線表示內部屬性

    @property
    def radius(self):
        """radius 的 getter"""
        return self._radius

    @radius.setter
    def radius(self, value):
        """radius 的 setter"""
        if value < 0:
            raise ValueError("半徑不能為負數")
        self._radius = value

    @property
    def diameter(self):
        """直徑（唯讀屬性）"""
        return self._radius * 2

    @property
    def area(self):
        """面積（唯讀屬性）"""
        return math.pi * self._radius ** 2

# 使用
circle = Circle(5)

# 像屬性一樣存取（實際上呼叫 getter）
print(f"radius: {circle.radius}")
print(f"diameter: {circle.diameter}")
print(f"area: {circle.area:.2f}")

# 像屬性一樣設定（實際上呼叫 setter）
circle.radius = 10
print(f"\n修改後 diameter: {circle.diameter}")

In [ ]:
# 設定無效值會觸發 setter 中的驗證
try:
    circle.radius = -5
except ValueError as e:
    print(f"錯誤: {e}")

### 計算屬性與快取

In [ ]:
# 使用 functools.cached_property（Python 3.8+）
from functools import cached_property

class Square:
    def __init__(self, side):
        self.side = side

    @cached_property
    def area(self):
        """只計算一次的屬性"""
        print("計算面積...")
        return self.side ** 2

square = Square(5)
print(f"第一次存取: {square.area}")  # 計算面積... 25
print(f"第二次存取: {square.area}")  # 25（使用快取）

---

## 7.2 繼承

### 基本繼承

In [ ]:
# 父類別（基礎類別）
class Animal:
    def __init__(self, name):
        self.name = name

    def speak(self):
        return "Some sound"

    def info(self):
        return f"I'm {self.name}"

# 子類別（衍生類別）
class Dog(Animal):
    def __init__(self, name, breed):
        super().__init__(name)  # 呼叫父類別的 __init__
        self.breed = breed

    def speak(self):  # 覆寫父類別方法
        return "Woof!"

    def fetch(self):  # 新增方法
        return f"{self.name} is fetching!"

class Cat(Animal):
    def speak(self):
        return "Meow!"

# 使用
dog = Dog("Buddy", "Golden Retriever")
cat = Cat("Whiskers")

print(f"dog.name: {dog.name}")        # 繼承的屬性
print(f"dog.breed: {dog.breed}")      # 自己的屬性
print(f"dog.speak(): {dog.speak()}")  # 覆寫的方法
print(f"dog.info(): {dog.info()}")    # 繼承的方法
print(f"dog.fetch(): {dog.fetch()}")  # 新增的方法
print(f"cat.speak(): {cat.speak()}")

### super() 函式

In [ ]:
class Parent:
    def __init__(self, name):
        self.name = name
        print(f"Parent.__init__ called: {name}")

    def greet(self):
        return f"Hello from Parent, I'm {self.name}"

class Child(Parent):
    def __init__(self, name, age):
        # 呼叫父類別的 __init__
        super().__init__(name)
        self.age = age
        print(f"Child.__init__ called: {name}, {age}")

    def greet(self):
        # 呼叫父類別的方法並擴展
        parent_greeting = super().greet()
        return f"{parent_greeting}, and I'm {self.age} years old"

child = Child("Alice", 10)
print(f"\n{child.greet()}")

### 多重繼承

Python 支援多重繼承（JavaScript 不支援）：

In [ ]:
class Flyable:
    def fly(self):
        return "Flying!"

class Swimmable:
    def swim(self):
        return "Swimming!"

class Walkable:
    def walk(self):
        return "Walking!"

# 多重繼承
class Duck(Flyable, Swimmable, Walkable):
    def __init__(self, name):
        self.name = name

    def quack(self):
        return "Quack!"

duck = Duck("Donald")
print(f"duck.fly(): {duck.fly()}")
print(f"duck.swim(): {duck.swim()}")
print(f"duck.walk(): {duck.walk()}")
print(f"duck.quack(): {duck.quack()}")

In [ ]:
# 檢查繼承
print(f"isinstance(duck, Flyable): {isinstance(duck, Flyable)}")
print(f"isinstance(duck, Swimmable): {isinstance(duck, Swimmable)}")
print(f"issubclass(Duck, Flyable): {issubclass(Duck, Flyable)}")

### 方法解析順序（MRO）

多重繼承時，Python 使用 C3 線性化演算法決定方法搜尋順序：

In [ ]:
class A:
    def method(self):
        return "A"

class B(A):
    def method(self):
        return "B"

class C(A):
    def method(self):
        return "C"

class D(B, C):
    pass

# D 繼承 B 和 C，兩者都繼承 A
d = D()
print(f"d.method(): {d.method()}")  # "B"（先找到 B）

# 查看方法解析順序
print(f"D.__mro__: {D.__mro__}")

### Mixin 模式

使用多重繼承實現功能組合：

In [ ]:
import json

# Mixin 類別：提供特定功能
class JSONMixin:
    """提供 JSON 序列化功能"""
    def to_json(self):
        return json.dumps(self.__dict__)

    @classmethod
    def from_json(cls, json_str):
        data = json.loads(json_str)
        return cls(**data)

class PrintableMixin:
    """提供友善的字串表示"""
    def __str__(self):
        attrs = ", ".join(f"{k}={v!r}" for k, v in self.__dict__.items())
        return f"{self.__class__.__name__}({attrs})"

    def __repr__(self):
        return self.__str__()

# 組合使用
class Person(JSONMixin, PrintableMixin):
    def __init__(self, name, age):
        self.name = name
        self.age = age

person1 = Person("Alice", 30)
print(person1)
print(f"to_json: {person1.to_json()}")

# 從 JSON 還原
json_str = '{"name": "Bob", "age": 25}'
person2 = Person.from_json(json_str)
print(f"from_json: {person2}")

---

## 7.3 存取控制

Python 沒有真正的 private/protected 關鍵字，而是使用命名慣例：

### 命名慣例

In [ ]:
class MyClass:
    def __init__(self):
        self.public = "公開"           # 公開屬性
        self._protected = "保護"       # 慣例上的 protected
        self.__private = "私有"        # Name mangling

    def public_method(self):
        """公開方法"""
        return "public method"

    def _protected_method(self):
        """慣例上的 protected 方法"""
        return "protected method"

    def __private_method(self):
        """私有方法（name mangling）"""
        return "private method"

obj = MyClass()

# 公開：正常存取
print(f"obj.public: {obj.public}")
print(f"obj.public_method(): {obj.public_method()}")

# Protected：可以存取，但慣例上不應該
print(f"obj._protected: {obj._protected}")
print(f"obj._protected_method(): {obj._protected_method()}")

In [ ]:
# Private：被 name mangling 處理
try:
    print(obj.__private)
except AttributeError as e:
    print(f"直接存取 __private 失敗: {e}")

# 但實際上還是可以存取（透過 mangled 名稱）
print(f"obj._MyClass__private: {obj._MyClass__private}")
print(f"obj._MyClass__private_method(): {obj._MyClass__private_method()}")

### 存取控制對照表

| 命名 | 存取範圍 | 說明 |
|------|----------|------|
| `name` | 公開 | 任何地方都可存取 |
| `_name` | Protected（慣例） | 可存取，但慣例上只在類別內部和子類別使用 |
| `__name` | Private（name mangling） | 變成 `_ClassName__name`，較難存取 |
| `__name__` | 特殊方法 | Python 內部使用，如 `__init__`、`__str__` |

### 使用 @property 實現封裝

In [ ]:
class BankAccount:
    def __init__(self, owner, balance=0):
        self._owner = owner
        self._balance = balance
        self._transactions = []

    @property
    def owner(self):
        """唯讀屬性"""
        return self._owner

    @property
    def balance(self):
        """唯讀屬性"""
        return self._balance

    @property
    def transactions(self):
        """回傳交易記錄的複本，防止外部修改"""
        return self._transactions.copy()

    def deposit(self, amount):
        """存款"""
        if amount <= 0:
            raise ValueError("存款金額必須為正數")
        self._balance += amount
        self._transactions.append(f"存款: +{amount}")

    def withdraw(self, amount):
        """提款"""
        if amount <= 0:
            raise ValueError("提款金額必須為正數")
        if amount > self._balance:
            raise ValueError("餘額不足")
        self._balance -= amount
        self._transactions.append(f"提款: -{amount}")

# 使用
account = BankAccount("Alice", 1000)
account.deposit(500)
account.withdraw(200)

print(f"owner: {account.owner}")
print(f"balance: {account.balance}")
print(f"transactions: {account.transactions}")

### __slots__ 限制屬性

In [ ]:
class Point:
    __slots__ = ['x', 'y']  # 只允許這些屬性

    def __init__(self, x, y):
        self.x = x
        self.y = y

point = Point(10, 20)
print(f"point.x: {point.x}")
print(f"point.y: {point.y}")

# 無法新增其他屬性
try:
    point.z = 30
except AttributeError as e:
    print(f"新增屬性失敗: {e}")

# __slots__ 的優點：
# 1. 節省記憶體（不需要 __dict__）
# 2. 存取速度更快
# 3. 防止意外新增屬性

---

## 7.4 特殊方法（Magic Methods）

特殊方法（也稱為 Dunder Methods）讓類別可以與 Python 的內建操作整合。

### 字串表示

In [ ]:
class Person:
    def __init__(self, name, age):
        self.name = name
        self.age = age

    def __str__(self):
        """給使用者看的字串（print、str()）"""
        return f"{self.name}, {self.age} 歲"

    def __repr__(self):
        """給開發者看的字串（除錯、互動式環境）"""
        return f"Person(name={self.name!r}, age={self.age!r})"

person = Person("Alice", 30)

print(person)         # 使用 __str__
print(str(person))
print(repr(person))   # 使用 __repr__

# 在列表中會使用 __repr__
people = [Person("Alice", 30), Person("Bob", 25)]
print(people)

### 比較運算子

In [ ]:
from functools import total_ordering

@total_ordering
class Money:
    def __init__(self, amount, currency="TWD"):
        self.amount = amount
        self.currency = currency

    def __eq__(self, other):
        """等於 =="""
        if not isinstance(other, Money):
            return NotImplemented
        return self.amount == other.amount and self.currency == other.currency

    def __lt__(self, other):
        """小於 <"""
        if not isinstance(other, Money) or self.currency != other.currency:
            return NotImplemented
        return self.amount < other.amount

    def __repr__(self):
        return f"Money({self.amount}, {self.currency!r})"

# 使用
m1 = Money(100)
m2 = Money(100)
m3 = Money(200)

print(f"m1 == m2: {m1 == m2}")
print(f"m1 < m3: {m1 < m3}")
print(f"m3 > m1: {m3 > m1}")
print(f"m1 <= m2: {m1 <= m2}")

### 算術運算子

In [ ]:
class Vector:
    def __init__(self, x, y):
        self.x = x
        self.y = y

    def __add__(self, other):
        """加法 +"""
        if isinstance(other, Vector):
            return Vector(self.x + other.x, self.y + other.y)
        return NotImplemented

    def __sub__(self, other):
        """減法 -"""
        if isinstance(other, Vector):
            return Vector(self.x - other.x, self.y - other.y)
        return NotImplemented

    def __mul__(self, scalar):
        """乘法 *（向量 * 數字）"""
        if isinstance(scalar, (int, float)):
            return Vector(self.x * scalar, self.y * scalar)
        return NotImplemented

    def __rmul__(self, scalar):
        """反向乘法（數字 * 向量）"""
        return self.__mul__(scalar)

    def __neg__(self):
        """負號 -v"""
        return Vector(-self.x, -self.y)

    def __abs__(self):
        """絕對值 abs(v)"""
        return (self.x ** 2 + self.y ** 2) ** 0.5

    def __repr__(self):
        return f"Vector({self.x}, {self.y})"

# 使用
v1 = Vector(3, 4)
v2 = Vector(1, 2)

print(f"v1 + v2 = {v1 + v2}")
print(f"v1 - v2 = {v1 - v2}")
print(f"v1 * 2 = {v1 * 2}")
print(f"3 * v1 = {3 * v1}")
print(f"-v1 = {-v1}")
print(f"abs(v1) = {abs(v1)}")

### 容器方法

In [ ]:
class Playlist:
    def __init__(self, name):
        self.name = name
        self._songs = []

    def __len__(self):
        """len(playlist)"""
        return len(self._songs)

    def __getitem__(self, index):
        """playlist[index] 或 playlist[start:end]"""
        return self._songs[index]

    def __setitem__(self, index, value):
        """playlist[index] = value"""
        self._songs[index] = value

    def __delitem__(self, index):
        """del playlist[index]"""
        del self._songs[index]

    def __contains__(self, item):
        """item in playlist"""
        return item in self._songs

    def __iter__(self):
        """for song in playlist"""
        return iter(self._songs)

    def add(self, song):
        self._songs.append(song)

    def __repr__(self):
        return f"Playlist({self.name!r}, {len(self)} songs)"

# 使用
playlist = Playlist("My Favorites")
playlist.add("Song A")
playlist.add("Song B")
playlist.add("Song C")

print(f"len(playlist): {len(playlist)}")
print(f"playlist[0]: {playlist[0]}")
print(f"playlist[1:]: {playlist[1:]}")
print(f"'Song A' in playlist: {'Song A' in playlist}")

playlist[0] = "Song A (Remix)"
print(f"\n修改後 playlist[0]: {playlist[0]}")

print("\n遍歷:")
for song in playlist:
    print(f"  {song}")

### 可呼叫物件

In [ ]:
class Multiplier:
    def __init__(self, factor):
        self.factor = factor

    def __call__(self, value):
        """讓實例可以像函式一樣呼叫"""
        return value * self.factor

# 使用
double = Multiplier(2)
triple = Multiplier(3)

print(f"double(5): {double(5)}")
print(f"triple(5): {triple(5)}")
print(f"callable(double): {callable(double)}")

In [ ]:
# 實際應用：帶狀態的函式
class Counter:
    def __init__(self):
        self.count = 0

    def __call__(self):
        self.count += 1
        return self.count

counter = Counter()
print(counter())  # 1
print(counter())  # 2
print(counter())  # 3

### Context Manager（with 語句）

In [ ]:
class Timer:
    """計時 Context Manager"""
    def __init__(self, name="Timer"):
        self.name = name

    def __enter__(self):
        """進入 with 區塊時呼叫"""
        import time
        self.start = time.time()
        print(f"[{self.name}] 開始計時")
        return self

    def __exit__(self, exc_type, exc_val, exc_tb):
        """離開 with 區塊時呼叫"""
        import time
        self.end = time.time()
        self.elapsed = self.end - self.start
        print(f"[{self.name}] 結束計時，耗時: {self.elapsed:.4f} 秒")
        return False  # 不抑制例外

# 使用
import time

with Timer("測試") as t:
    # 模擬一些工作
    time.sleep(0.5)
    print("執行中...")

In [ ]:
# 實際應用：資料庫連線模擬
class DatabaseConnection:
    def __init__(self, connection_string):
        self.connection_string = connection_string
        self.connection = None

    def __enter__(self):
        print(f"連接資料庫: {self.connection_string}")
        self.connection = "Active Connection"
        return self

    def __exit__(self, exc_type, exc_val, exc_tb):
        print("關閉資料庫連線")
        self.connection = None
        return False

    def query(self, sql):
        print(f"執行查詢: {sql}")
        return ["result1", "result2"]

with DatabaseConnection("localhost:5432") as db:
    results = db.query("SELECT * FROM users")
    print(f"查詢結果: {results}")

### 特殊方法總覽

| 方法 | 觸發方式 | 說明 |
|------|----------|------|
| `__init__` | `ClassName()` | 初始化 |
| `__str__` | `str()`, `print()` | 使用者友善字串 |
| `__repr__` | `repr()` | 開發者字串 |
| `__len__` | `len()` | 長度 |
| `__getitem__` | `obj[key]` | 取得項目 |
| `__setitem__` | `obj[key] = value` | 設定項目 |
| `__contains__` | `in` | 成員檢查 |
| `__iter__` | `iter()`, `for` | 迭代 |
| `__call__` | `obj()` | 呼叫物件 |
| `__enter__` | `with` | 進入上下文 |
| `__exit__` | `with` | 離開上下文 |
| `__eq__` | `==` | 相等 |
| `__lt__` | `<` | 小於 |
| `__add__` | `+` | 加法 |
| `__sub__` | `-` | 減法 |
| `__mul__` | `*` | 乘法 |

---

## 7.5 抽象基礎類別（ABC）

In [ ]:
from abc import ABC, abstractmethod
import math

class Shape(ABC):
    """抽象基礎類別"""

    @abstractmethod
    def area(self):
        """子類別必須實作"""
        pass

    @abstractmethod
    def perimeter(self):
        """子類別必須實作"""
        pass

    def description(self):
        """一般方法（可選擇覆寫）"""
        return f"這是一個 {self.__class__.__name__}"

# 不能直接實例化抽象類別
try:
    shape = Shape()
except TypeError as e:
    print(f"無法實例化抽象類別: {e}")

In [ ]:
class Rectangle(Shape):
    def __init__(self, width, height):
        self.width = width
        self.height = height

    def area(self):
        return self.width * self.height

    def perimeter(self):
        return 2 * (self.width + self.height)

class Circle(Shape):
    def __init__(self, radius):
        self.radius = radius

    def area(self):
        return math.pi * self.radius ** 2

    def perimeter(self):
        return 2 * math.pi * self.radius

# 使用
rect = Rectangle(10, 5)
circle = Circle(7)

print(f"Rectangle:")
print(f"  area: {rect.area()}")
print(f"  perimeter: {rect.perimeter()}")
print(f"  description: {rect.description()}")

print(f"\nCircle:")
print(f"  area: {circle.area():.2f}")
print(f"  perimeter: {circle.perimeter():.2f}")

---

## 練習題

### 練習 1：銀行帳戶類別

建立一個 `BankAccount` 類別，具有以下功能：

In [ ]:
class BankAccount:
    # 你的程式碼
    pass

# 預期使用方式
# account = BankAccount("Alice", 1000)
# account.deposit(500)
# account.withdraw(200)
# print(account.balance)      # 1300
# print(account.owner)        # "Alice"
# print(account)              # "BankAccount(owner='Alice', balance=1300)"

In [ ]:
# 參考解答
class BankAccount:
    def __init__(self, owner, balance=0):
        self._owner = owner
        self._balance = balance

    @property
    def owner(self):
        return self._owner

    @property
    def balance(self):
        return self._balance

    def deposit(self, amount):
        if amount <= 0:
            raise ValueError("存款金額必須為正數")
        self._balance += amount

    def withdraw(self, amount):
        if amount <= 0:
            raise ValueError("提款金額必須為正數")
        if amount > self._balance:
            raise ValueError("餘額不足")
        self._balance -= amount

    def __repr__(self):
        return f"BankAccount(owner={self._owner!r}, balance={self._balance})"

# 測試
account = BankAccount("Alice", 1000)
account.deposit(500)
account.withdraw(200)
print(f"balance: {account.balance}")
print(f"owner: {account.owner}")
print(account)

### 練習 2：自訂串列類別

建立一個 `MyList` 類別，支援 `+` 運算子合併兩個列表：

In [ ]:
class MyList:
    # 你的程式碼
    pass

# 預期使用方式
# my_list = MyList([1, 2, 3])
# print(len(my_list))         # 3
# print(my_list[0])           # 1
# print(2 in my_list)         # True
# my_list.append(4)
# print(list(my_list))        # [1, 2, 3, 4]
#
# my_list2 = MyList([5, 6])
# combined = my_list + my_list2
# print(list(combined))       # [1, 2, 3, 4, 5, 6]

In [ ]:
# 參考解答
class MyList:
    def __init__(self, items=None):
        self._items = list(items) if items else []

    def __len__(self):
        return len(self._items)

    def __getitem__(self, index):
        return self._items[index]

    def __contains__(self, item):
        return item in self._items

    def __iter__(self):
        return iter(self._items)

    def __add__(self, other):
        if isinstance(other, MyList):
            return MyList(self._items + other._items)
        return NotImplemented

    def append(self, item):
        self._items.append(item)

    def __repr__(self):
        return f"MyList({self._items})"

# 測試
my_list = MyList([1, 2, 3])
print(f"len: {len(my_list)}")
print(f"my_list[0]: {my_list[0]}")
print(f"2 in my_list: {2 in my_list}")

my_list.append(4)
print(f"after append: {list(my_list)}")

my_list2 = MyList([5, 6])
combined = my_list + my_list2
print(f"combined: {list(combined)}")

---

## 小結

### JavaScript vs Python 類別對照

| 特性 | JavaScript | Python |
|------|------------|--------|
| 類別定義 | `class Name {}` | `class Name:` |
| 建構子 | `constructor()` | `__init__(self)` |
| 實例參考 | `this` | `self` |
| 建立實例 | `new Class()` | `Class()` |
| 繼承 | `extends` | `class Child(Parent):` |
| 呼叫父類別 | `super.method()` | `super().method()` |
| 靜態方法 | `static method()` | `@staticmethod` |
| Getter/Setter | `get/set` | `@property` |
| 私有 | `#field` | `__field`（name mangling） |

### 存取修飾符

| Python | 說明 | JavaScript 對應 |
|--------|------|----------------|
| `name` | 公開 | `name` |
| `_name` | Protected（慣例） | 無直接對應 |
| `__name` | Private（name mangling） | `#name` |

### 常用特殊方法

| 類別 | 方法 |
|------|------|
| 基本 | `__init__`, `__str__`, `__repr__` |
| 比較 | `__eq__`, `__lt__`, `__gt__`, `__le__`, `__ge__` |
| 算術 | `__add__`, `__sub__`, `__mul__`, `__truediv__` |
| 容器 | `__len__`, `__getitem__`, `__setitem__`, `__contains__`, `__iter__` |
| 其他 | `__call__`, `__enter__`, `__exit__`, `__hash__` |